In [1]:
import numpy as np

# Question 1

## 1a)

In [40]:
S0 = 100
sigma = 0.3
r = b = 0.05
T = 2


def Sn(T, n, b, sigma, j):
    """
    Vector of all possible prices for the option of maturity T, at time j.
    """
    hn = T/n
    u = np.exp(b*hn + sigma*np.sqrt(hn)),
    d = np.exp(b*hn - sigma*np.sqrt(hn))
    indices = np.arange(0, j)
    return S0*np.power(u, j-indices)*np.power(d, indices)

n = 3
snj = Sn(T, n, b, sigma, 2)
print(snj)

[174.46685854 106.89391057]


## 1b)

In [65]:
def Payoffn(T, n, b, sigma, K):
    """
    Compute the Payoff of an European call option with maturity T,
    and strike K.
    """
    ## current time is maturity, j=T
    snT = Sn(T, n, b, sigma, T)
    delta = snT - K
    vals = np.stack([delta, np.zeros_like(delta)])
    return np.max(vals, 0) # Maximum selon la première dimension, donc les valeurs max(snT-K, 0)

K = 110
Payoffn(T, 5, b, sigma, K)

array([42.11601788,  0.        ])

## 1c)

In [66]:
def Calln(T, n, r, b, sigma, K):
    """
    Compute the price of an European call option
    """
    payoff = Payoffn(T, n, b, sigma, K)
    # Price is the adjusted expected value of the payoff
    return np.exp(-r*T)*np.mean(payoff)

Calln(T, n, r, b, sigma, K)

29.16601291616392

## 1d)

In [ ]:
def Deltan(T, n, r, b, sigma, K, j):
    """
    """
    hn = T/n
    tnj = j*hn
    return